# Python web scraper script

1. installing requirements

In [5]:
# import sys
# # you can also directly run this command in CLI 
# # installing pandas
# !{sys.executable} -m pip install pandas
# # installing bs4
# !{sys.executable} -m pip install bs4
# # for exporting in excel sheet
# !{sys.executable} -m pip install openpyxl
# !{sys.executable} -m pip install lxml




2. Importing packages and libraries

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import lxml
import requests, time, datetime, re, random

In [7]:
url = "https://en.m.wikipedia.org/wiki/List_of_best-selling_Nintendo_Switch_video_games"
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
response = requests.get(url, headers=headers)
# check if the status is ok 
print(response.status_code)
# print(response.text)

200


In [ ]:
soup = BeautifulSoup(response.text, 'lxml')
table_html = soup.find('table' , {'class':"wikitable"})
# table_html

In [9]:
table_dataframe = pd.read_html(str(table_html))[0]
table_dataframe

,Rank,Title,Copies sold,As of,Release date[a],Genre(s),Developer(s),Publisher(s)
0,1,Mario Kart 8 Deluxe,53.79 million[4],"March 31, 2023","April 28, 2017",Kart racing,Nintendo EPD,Nintendo
1,2,Animal Crossing: New Horizons,42.21 million[4],"March 31, 2023","March 20, 2020",Social simulation,Nintendo EPD,Nintendo
2,3,Super Smash Bros. Ultimate,31.09 million[4],"March 31, 2023","December 7, 2018",Fighting,Bandai Namco StudiosSora Ltd.,Nintendo
3,4,The Legend of Zelda: Breath of the Wild,29.81 million[4],"March 31, 2023","March 3, 2017",Action-adventure,Nintendo EPD,Nintendo
4,5,Pokémon Sword and Shield,25.82 million[4],"March 31, 2023","November 15, 2019",Role-playing,Game Freak,The Pokémon CompanyNintendo
...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,1 million[25],"September 8, 2020","December 20, 2018",Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo
74,72,Fitness Boxing 2: Rhythm and Exercise,1 million[26],"December 9, 2021","December 4, 2020",Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo
75,72,Shin Megami Tensei V,1 million[27],"April 18, 2022","November 11, 2021",Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo
76,72,Story of Seasons: Pioneers of Olive Town,1 million[28],"November 18, 2021","February 25, 2021",Simulationrole-playing,Marvelous,Xseed Games


### Formatting the price, dates ,url

In [39]:
# function to convert the string copies sold value to float
def convert_to_float(copies_sold):
    #  remove the string value using regex
    clean_value = re.sub(r'million(\[[a-z]*[0-9]*\])*','',copies_sold)
    return float(clean_value)*1_000_000

# print(convert_to_float("1 million[29]"))

def format_date(date):
    current_format = "%B %d, %Y"
    desired_format = "%m/%d/%Y"
    date_string = datetime.datetime.strptime(date,current_format)

    return date_string.strftime(desired_format)

# format_date("March 31, 2023")

# getting reviews from metacentric website
def create_reviews_url(link):
    clean_game_title = re.sub(r'[:,;.\'?\[\]]','',link).replace(' ','-').replace('--', '-').replace('é','e').lower()
    return f'https://www.metacritic.com/game/switch/{clean_game_title}/details'


create_reviews_url("Mario Kart 8 Deluxe")


'https://www.metacritic.com/game/switch/mario-kart-8-deluxe/details'

In [11]:
sales_formatted_df = table_dataframe.assign(
    
    As_of = lambda df: df ['As of'].map(lambda values: format_date(values)),
    Release_date = lambda df: df ['Release date[a]'].map((lambda values: format_date(values))),
    Copies_sold = lambda df: df['Copies sold'].map(lambda values: convert_to_float(values)),
    Review_link = lambda df: df['Title'].map(lambda values: create_reviews_url(values)),

).drop(columns=['As of','Copies sold','Release date[a]'], axis=1)

sales_formatted_df

,Rank,Title,Genre(s),Developer(s),Publisher(s),As_of,Release_date,Copies_sold,Review_link
0,1,Mario Kart 8 Deluxe,Kart racing,Nintendo EPD,Nintendo,03/31/2023,04/28/2017,53790000.0,https://www.metacritic.com/game/switch/mario-k...
1,2,Animal Crossing: New Horizons,Social simulation,Nintendo EPD,Nintendo,03/31/2023,03/20/2020,42210000.0,https://www.metacritic.com/game/switch/animal-...
2,3,Super Smash Bros. Ultimate,Fighting,Bandai Namco StudiosSora Ltd.,Nintendo,03/31/2023,12/07/2018,31090000.0,https://www.metacritic.com/game/switch/super-s...
3,4,The Legend of Zelda: Breath of the Wild,Action-adventure,Nintendo EPD,Nintendo,03/31/2023,03/03/2017,29810000.0,https://www.metacritic.com/game/switch/the-leg...
4,5,Pokémon Sword and Shield,Role-playing,Game Freak,The Pokémon CompanyNintendo,03/31/2023,11/15/2019,25820000.0,https://www.metacritic.com/game/switch/pokemon...
...,...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,09/08/2020,12/20/2018,1000000.0,https://www.metacritic.com/game/switch/fitness...
74,72,Fitness Boxing 2: Rhythm and Exercise,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,12/09/2021,12/04/2020,1000000.0,https://www.metacritic.com/game/switch/fitness...
75,72,Shin Megami Tensei V,Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo,04/18/2022,11/11/2021,1000000.0,https://www.metacritic.com/game/switch/shin-me...
76,72,Story of Seasons: Pioneers of Olive Town,Simulationrole-playing,Marvelous,Xseed Games,11/18/2021,02/25/2021,1000000.0,https://www.metacritic.com/game/switch/story-o...


### Getting from Metacentric website

In [ ]:
# function to get info from metacentric website
def getMetacentricData(link):
    response_metacentric = requests.get(link,headers=headers)
    soup_metacentric = BeautifulSoup(response_metacentric.text,'html.parser')
    summary_element = soup_metacentric.find('div', class_='section product_details')
    game_description = summary_element.find('span',class_='data').get_text()
    score_metacentric = soup_metacentric.find('div', class_='score_summary metascore_summary').find('span', itemprop='ratingValue').get_text()
    userReview_metacentric = soup_metacentric.find('div', class_='details side_details').find('a',class_='metascore_anchor').find('div').get_text()
    dictiornaryreview = {
        'Review_link' : link,
        'description' : game_description,
        'Meta Score' : score_metacentric,
        'User-Review' : userReview_metacentric,
    }
    return dictiornaryreview

sources = sales_formatted_df['Review_link'].tolist()

# Initialize the dictionary to store the data from all links
all_data = []

# Iterate through the links and call the function for each link
for item in sources:
    try:
        data = getMetacentricData(item)
    except:
        data = {
        'Review_link' : item,
        'description' : '',
        'Meta Score' : '',
        'User-Review' : '',
        }
    all_data.append(data)
    time.sleep(2)



### Converting the list to panda dataframe

In [ ]:

metacentricReviewsdf = pd.DataFrame(all_data)

In [65]:
merged_df = pd.merge(sales_formatted_df, metacentricReviewsdf, on='Review_link')
merged_df

,Rank,Title,Genre(s),Developer(s),Publisher(s),As_of,Release_date,Copies_sold,Review_link,description,Meta Score,User-Review
0,1,Mario Kart 8 Deluxe,Kart racing,Nintendo EPD,Nintendo,03/31/2023,04/28/2017,53790000.0,https://www.metacritic.com/game/switch/mario-k...,Race and battle your friends in the definitive...,92,8.6
1,2,Animal Crossing: New Horizons,Social simulation,Nintendo EPD,Nintendo,03/31/2023,03/20/2020,42210000.0,https://www.metacritic.com/game/switch/animal-...,If the hustle and bustle of modern life’s got ...,90,5.6
2,3,Super Smash Bros. Ultimate,Fighting,Bandai Namco StudiosSora Ltd.,Nintendo,03/31/2023,12/07/2018,31090000.0,https://www.metacritic.com/game/switch/super-s...,"Inklings from the Splatoon series, as well as ...",93,8.6
3,4,The Legend of Zelda: Breath of the Wild,Action-adventure,Nintendo EPD,Nintendo,03/31/2023,03/03/2017,29810000.0,https://www.metacritic.com/game/switch/the-leg...,Forget everything you know about The Legend of...,97,8.7
4,5,Pokémon Sword and Shield,Role-playing,Game Freak,The Pokémon CompanyNintendo,03/31/2023,11/15/2019,25820000.0,https://www.metacritic.com/game/switch/pokemon...,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
73,72,Fitness Boxing,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,09/08/2020,12/20/2018,1000000.0,https://www.metacritic.com/game/switch/fitness...,"Get off the couch and get moving with fun, box...",66,6.5
74,72,Fitness Boxing 2: Rhythm and Exercise,Exergamerhythm,Imagineer,JP: ImagineerNA/PAL: Nintendo,12/09/2021,12/04/2020,1000000.0,https://www.metacritic.com/game/switch/fitness...,,,
75,72,Shin Megami Tensei V,Role-playing,Atlus,JP: AtlusNA: SegaPAL: Nintendo,04/18/2022,11/11/2021,1000000.0,https://www.metacritic.com/game/switch/shin-me...,The ambitions of god and human clash amidst th...,84,7.5
76,72,Story of Seasons: Pioneers of Olive Town,Simulationrole-playing,Marvelous,Xseed Games,11/18/2021,02/25/2021,1000000.0,https://www.metacritic.com/game/switch/story-o...,Inspired by tales of their grandfather's pione...,71,7.1


In [66]:
# merged_df.to_excel('nintendo_switch_games.xlsx', index=False)
merged_df.to_excel('nintendo_switch_games.csv', index=False)